In [1]:
import warnings
warnings.filterwarnings("ignore")
import os

os.sys.path.insert(0, '../script')

from config import *
from webnlg import *

In [2]:
my_models_paths = !ls -d -1 ../data/models/*.txt

my_models_paths

['../data/models/5 - Model - Template Based - roots_4e9245cbbc05e4fdf0de404fbada20d8294127b2.txt']

In [3]:
models_paths = my_models_paths + SUBMISSIONS_FILEPATHS
models_paths

['../data/models/5 - Model - Template Based - roots_4e9245cbbc05e4fdf0de404fbada20d8294127b2.txt',
 '../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt',
 '../data/webnlg2017/submissions/melbourne/final_result.txt',
 '../data/webnlg2017/submissions/pkuwriter/PKUWriter_results.txt',
 '../data/webnlg2017/submissions/tilburg/nmt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/smt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/template_test.out.ordered',
 '../data/webnlg2017/submissions/uit-danglt-clnlp/Submission-UIT-DANGNT-CLNLP.txt',
 '../data/webnlg2017/submissions/upf/UPF_All_sent_final.txt',
 '../data/webnlg2017/submissions/baseline_sorted.txt']

In [4]:
model_results = {}

for model_path in models_paths:
    
    with open(model_path) as f:
        
        model_results[model_path] = [line[:-1] for line in f]

In [5]:
import pandas as pd
pd.set_option('max_colwidth', 1000)
from nltk.translate.bleu_score import sentence_bleu

lex = WebNLGCorpus.load("test_with_lex")

s_multiwordexpressions = lex.mdf.m_subject.str.lower().str.replace('_', ' ').str.split().tolist()
o_multiwordexpressions = lex.mdf.m_object.str.lower().str.replace('_', ' ').str.split().tolist()

all_multiwordexpressions = s_multiwordexpressions + o_multiwordexpressions
all_multiwordexpressions = list(set([tuple(s) for s in all_multiwordexpressions]))

from nltk.tokenize.mwe import MWETokenizer

t = MWETokenizer(all_multiwordexpressions)

from textacy.preprocess import preprocess_text

def preprocess(ref):
    
    ref = preprocess_text(ref, no_accents=True, lowercase=True, no_punct=True)
    
    return t.tokenize(ref.split())


def print_entry_and_models(e):
    
    print(e)
    print('\n')
    
    idx = int(e.idx().split('_')[1])
    
    references = [preprocess(ref) for ref in e.ldf.ltext.tolist()]
    
    texts = []
    bleus = []
    
    for model_path in models_paths:
        
        model_text = model_results[model_path][idx]
        
        bleu = sentence_bleu(references, preprocess(model_text))
        
        texts.append(model_text)
        bleus.append(bleu)
        
    models_names = [model_path.split('/')[-1] for model_path in models_paths]
        
    return pd.DataFrame({'models': models_names,
                         'texts': texts,
                         'bleu': bleus}).sort_values('bleu', ascending=False)

In [6]:
test_with_lex = WebNLGCorpus.load('test_with_lex')

In [13]:
e = test_with_lex.sample()

df = print_entry_and_models(e)

Triple info: {'category': 'Building', 'eid': 'Id382', 'idx': '0_381', 'ntriples': 2}

	Modified triples:

3Arena | location | Dublin
3Arena | architect | Populous_(company)


	Lexicalizations:

The 3Arena, which was designed by the architect company Populous, is located in Dublin.
The 3Arena, designed by Populous, is located in Dublin.
3Arena is located in Dublin, which was built by the company Populous.




In [14]:
df

,bleu,models,texts
1,8.118741e-01,ADAPTcentreWebNLGsubmission.txt,"the 3arena , designed by the company populous , is located in dublin ."
3,6.053287e-01,PKUWriter_results.txt,3arena is located in dublin and was designed by the architect populous ( company ) .
8,6.313993e-78,UPF_All_sent_final.txt,"3arena , which the company populous designed , is in dublin ."
5,5.572079e-78,smt_test.out.ordered,architect was the company populous . 3arena in dublin
7,4.601359e-78,Submission-UIT-DANGNT-CLNLP.txt,"populous ( company ) was the architect of the 3arena , which is located at the dublin ."
2,1.030133e-154,final_result.txt,populous is the architect of 3arena in dublin .
0,9.905379e-155,5 - Model - Template Based - roots_4e9245cbbc05e4fdf0de404fbada20d8294127b2.txt,"3Arena is Dublin. The architect of , is Populous (company)."
4,9.587021e-155,nmt_test.out.ordered,populous was the architect of 3arena which is located at '' dublin '' .
9,7.827317e-155,baseline_sorted.txt,"the architect of 3arena , dublin , was populous ( company ) ."
6,5.219636e-232,template_test.out.ordered,"populous designed 3arena , dublin ."
